In [1]:
import sys
sys.path.append("../../")
from STK_Sim import *

Filename = 'AERO_402_Further_Assessment'

stk_object = STK_Simulation(False,Filename)

In [2]:
# Comparing high data rate w/ no interpolation to low data rate w/ interpolation data

stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
duration = datetime.timedelta(days=1, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")

# Loading targets into stk from file.
stk_object.Target_Loader("../../Input_Files/Targets_File.txt")
stk_object.Satellite_Loader("../../Input_Files/Satellites_File.txt")

In [3]:
df = {}
df['Control Method'] = []
df['Angular_Momentum'] = []
df['Gravity_Grad_Torque'] = [] 
df['Total_Torque'] = []

stk_object.dt=600

stk_object.Update_Mass_Properties(M=250,I =  [[123.44,0,0],
                                              [0,88.88,0],
                                              [0,0,123.44]])

for Control_Method in ["NadirOrbit"]:
    for sat in stk_object.satellites:
        stk_object.root.ExecuteCommand(f"SetAttitude {sat.Path} Profile {Control_Method} Offset 0.0")

    Angular_Momentum_Body = stk_object.Get_Satellite_DP("Vectors(Body)/AngMomentum")
    Gravity_Grad_Torque_Body = stk_object.Get_Satellite_DP("Vectors(Body)/GrvGrdTorque")
    Total_Torque_Body = stk_object.Get_Satellite_DP("Vectors(Body)/TotalTorque")

    df['Control Method'].append(Control_Method)
    df['Total_Torque'].append(max([max(df.magnitude) for df in Total_Torque_Body]))
    df['Gravity_Grad_Torque'].append(max([max(df.magnitude) for df in Gravity_Grad_Torque_Body]))
    df['Angular_Momentum'].append(max([max(df.magnitude) for df in Angular_Momentum_Body]))

df = pd.DataFrame(df)
df

- Computing_Vectors(Body)/AngMomentum [==========] 5/5 [100%] in 0.3s (12.31/s) 
- Computing_Vectors(Body)/GrvGrdTorque [==========] 5/5 [100%] in 0.3s (18.11/s)
- Computing_Vectors(Body)/TotalTorque [==========] 5/5 [100%] in 0.3s (19.39/s) 


,Control Method,Angular_Momentum,Gravity_Grad_Torque,Total_Torque
0,NadirOrbit,0.133897,4.685504e-20,1.116862e-14
